# Import libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import numpy as np
import shutil, os
import datetime
import NodeLookup as lookup

In [1]:
df = pd.read_csv("data/train.csv")

# Creates folders for defferent categories categorized images from train data

In [2]:
for index, row in df.iterrows():
    print (row["image_name"], row["label"])
    if row["label"] == 0:
        shutil.copy('train/'+ row["image_name"], 'train/0')
    if row["label"] == 1:
        shutil.copy('train/'+ row["image_name"], 'train/1')
    if row["label"] == 2:
        shutil.copy('train/'+ row["image_name"], 'train/2')
    if row["label"] == 3:
        shutil.copy('train/'+ row["image_name"], 'train/3')
    if row["label"] == 4:
        shutil.copy('train/'+ row["image_name"], 'train/4')
    if row["label"] == 5:
        shutil.copy('train/'+ row["image_name"], 'train/5')  

# Use pretrained model from Tensorflow hub

In [5]:
# python retrain.py --tfhub_module  https://tfhub.dev/google/imagenet/inception_v3/classification/1 --image_dir train

In [ ]:
model_dir = 'imagenet'
read_mode = 'rb'
saved_graph = 'output_graph.pb'
label_path = 'output_labels.text'
graph_name = 'ImageNet'
softmax_tensor_name = graph_name + '/final_result:0'
image_tensor_name = graph_name + '/Placeholder:0'
log_path = 'logs' 

# Function to read image and convert to tensor

In [ ]:
def read_tensor_from_image_file(file_name,
                                input_height=331,
                                input_width=331,
                                input_mean=0,
                                input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(
        file_reader, channels=3, name="png_reader")
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(
        tf.image.decode_gif(file_reader, name="gif_reader"))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name="bmp_reader")
    else:
        image_reader = tf.image.decode_jpeg(
        file_reader, channels=3, name="jpeg_reader")


    float_caster = tf.cast(image_reader, tf.float32)

    dims_expander = tf.expand_dims(float_caster, 0)
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    result = sess.run(normalized)

    return result

In [ ]:
submitdf = pd.read_csv("data\sample_submission_CH2mq5Z.csv")

In [ ]:
num_predictions = 1

# Read trained model 

In [ ]:
with tf.gfile.GFile(saved_graph, read_mode) as graphFile:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(graphFile.read())
    _ = tf.import_graph_def(graph_def,name=graph_name)
    print('loaded model')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Make Predictions

In [ ]:
with tf.Session(config=config) as session:
    file_writer = tf.summary.FileWriter(log_path,session.graph)
    softmax_tensor = session.graph.get_tensor_by_name(softmax_tensor_name)
    
    for index, row in submitdf.iterrows():
        image_path = 'train/'+row["image_name"]
        image_data = read_tensor_from_image_file(image_path ,input_height=299,
                                input_width=299)
        
        image_tensor =  { image_tensor_name:image_data }
        predictions = session.run(softmax_tensor, image_tensor)
        predictions = np.squeeze(predictions)
        top_predictions = predictions.argsort()[-num_predictions:][::-1]
        for prediction in top_predictions:
            score = predictions[prediction]
            submitdf.at[index,'label'] = prediction
            print (row["image_name"], prediction)
            #print('{} == {:.2%}'.format(prediction,score))
        submitdf.to_csv("submission_part2.csv",index=False)